In [59]:
import math
import warnings
import numpy as np


NUMBER_OF_RECTANGLES = 100


def f(A, B, V, t):
    PQx = V.x - (A.x + ((B.x - A.x) * t))
    PQy = V.y - (A.y + ((B.y - A.y) * t))
    PQz = 0
    PQ = [PQx, PQy, PQz]
    AB = [
        B.x - A.x,
        B.y - A.y,
        B.z - A.z
    ]
    magnitudePQ = math.sqrt(PQx ** 2.0 + PQy ** 2.0 + PQz ** 2.0)
    forceMagnitude = 1 / magnitudePQ
    forceDirection = np.cross(PQ, [0, 0, 1]) / magnitudePQ
    forceVector = forceMagnitude * forceDirection
    return np.dot(forceVector, AB)


def integral(A, B, V):
    """Do the line integral"""
    numberofRectangles = NUMBER_OF_RECTANGLES
    startingt = 0.0
    endingt = 1.0
    width = (endingt - startingt) / numberofRectangles
    runningSum = 0.0
    for i in range(numberofRectangles):
        height = f(A, B, V, startingt + i * width)
        area = height * width
        runningSum += area
    return runningSum


class Vertex(object):
    """The three dimensional coordinates of a vertex"""
    def __init__(self, id, x, y, z):
        self.id = id
        self.x = x
        self.y = y
        self.z = z

    def __repr__(self):
        return str([self.x, self.y, self.z])

    def __eq__(self, other):
        return self.id == other.id

    def to_array(self):
        return np.array([self.x, self.y, self.z])

    def is_blocked(self, faces):
        """True when this vertex is blocked from view by faces"""
        for face in faces.values():
            if face.is_blocking(self):
                return True
        return False

    @staticmethod
    def load(filename):
        # vertices (vid, x, y, z)
        np_vertices = np.loadtxt(filename, delimiter="\t", skiprows=1,
                                 dtype=[('id', 'i4'), ('x', 'f8'), ('y', 'f8'), ('z', 'f8')])
        vertices = {}
        for id, x, y, z in np_vertices:
            vertices[id] = Vertex(id, x, y, z)
        return vertices


class Edge(object):
    """The vertex ID's of two vertices"""
    def __init__(self, id, v1, v2):
        self.id = id
        self.v1 = v1
        self.v2 = v2

    def __repr__(self):
        return str([self.v1, self.v2])

    def __iter__(self):
        return iter([self.v1, self.v2])

    def __eq__(self, other):
        return self.id == other.id

    @staticmethod
    def load(filename, vertices):
        # edge vertices (eid, vid, vid)
        np_edges = np.loadtxt(filename, delimiter="\t", skiprows=1, dtype="i4,i4,i4")

        edges = {}
        for id, v1, v2 in np_edges:
            edges[id] = Edge(id, vertices[v1], vertices[v2])
        return edges


class Face(object):
    def __init__(self, id, edges):
        """The list of edges must form a path"""
        self.id = id
        self.edges = edges
        self.vertices = Face._sort_vertices_into_path(edges)

    def __str__(self):
        return "\n".join(map(str, self.vertices))

    def is_behind(self, vertex):
        """True if the vertex is behind the plane of this face
        """
        # ok to use three vertices of the plane (Nige:28/7/15)
        V = vertex.to_array()
        A, B, C = [self.vertices[i].to_array() for i in [0, 1, 2]]
        AB = B - A
        AC = C - A
        ABxAC = np.cross(AB, AC)
        result = (((A[0] - V[0]) * (ABxAC[0]) + (A[1] - V[1]) * (ABxAC[1])) / (ABxAC[2])) + (A[2] - V[2])
        return result > 0

    def is_inside(self, vertex):
        """True if the vertex is blocked by this face

        Accumulate the integral of each edge in this face with the vertex. If the total is
        zero, then the vertex is visible.
        """
        try:
            total = 0.0
            for idx in range(len(self.vertices)):
                tegral = integral(self.vertices[idx], self.vertices[idx - 1], vertex)
                total += tegral
            result = abs(total) > 1 / NUMBER_OF_RECTANGLES  # relate the margin of error to number of rectangles
            return result
        except ZeroDivisionError:
            return False

    def is_blocking(self, vertex):
        """True when in line and is behind; whatever they mean"""
        return self.is_inside(vertex) and self.is_behind(vertex)

    @staticmethod
    def _sort_vertices_into_path(in_edges):
        """converts a series of edges into a path of vertices

        # assemble the vertices as a path around the edges of the face
        # by putting all the edges in a list and picking a start vertex
        # then find a cojoining edge, append the opposite vertex, and removing the edge,
        # until all the edges have been used.
        """
        edges = in_edges.copy()
        path = [edges[0].v1]
        edges.pop(0)
        while len(edges):
            for idx, edge in enumerate(edges):
                if edge.v1 == path[-1]:
                    path.append(edge.v2)
                    edges.pop(idx)
                    break
                elif edge.v2 == path[-1]:
                    path.append(edge.v1)
                    edges.pop(idx)
                    break
            else:
                raise Exception("These edges are expected to form a path: %s" % [e.id for e in in_edges])
        return path

    @staticmethod
    def load(filename, edges, vertices):

        # get the edges of the face (fid, eid, eid, eid, eid)
        # if the last edge is null indicate with a -1 edge id
        convertfunc = lambda x: -1 if x == b'NULL' else x
        np_faceedges = np.loadtxt(filename, delimiter="\t", skiprows=1,
                                  converters={4: convertfunc},
                                  dtype=[('id', 'i4'), ('edge1', 'i4'), ('edge2', 'i4'), ('edge3', 'i4'), ('edge4', 'i4')])

        # for every face
        faces = {}
        # get the edges
        for id, *edgeids in np_faceedges:
            try:
                face_edges = [edges[id] for id in edgeids if id >= 0]
                faces[id] = Face(id, face_edges)
            except KeyError:
                # warnings.warn("Face %s discarded because not all these edges are known (%s, %s, %s, %s)." % (id, eid1, eid2, eid3, eid4))
                pass
        return faces


In [60]:
vertices = Vertex.load('../headmesh/NVertices.txt')
edges = Edge.load('../headmesh/NEdgeVertices.txt', vertices)
faces = Face.load('../headmesh/NFaceEdges.txt', edges, vertices)
print("Loaded %s vertices, %s edges, %s faces." % (len(vertices), len(edges), len(faces)))

Loaded 775 vertices, 1520 edges, 752 faces.


In [51]:
A = Vertex('A', -4., -4., 0)
B = Vertex('B', 4., -4., 0)
C = Vertex('C', -4., 4., 0)
D = Vertex('D', 4., 4., 0)

edges = [Edge(1, A, B), Edge(2, B, C), Edge(3, C, D), Edge(4, D, A)]
face = Face(0, edges)
print(face)
print([v.id for v in face.vertices])

v1 = Vertex(1, 0, 0, 0)
print(v1.y, face.is_inside(v1), '\n')

v1 = Vertex(1, 0, 3.99, -0.1)
print(v1.y, face.is_inside(v1), '\n')

v1 = Vertex(1, 0, 4.00, -0.0)
print(v1.y, face.is_inside(v1), '\n')

v1 = Vertex(1, 0, 4.01, .1)
print(v1.y, face.is_inside(v1), '\n')

v1 = Vertex(1, 0, 4.10, -0.1)
print(v1.y, face.is_inside(v1), '\n')



[-4.0, -4.0, 0]
[4.0, 4.0, 0]
[-4.0, 4.0, 0]
[4.0, -4.0, 0]
['A', 'D', 'C', 'B']
0 False 

3.99 True 

4.0 False 

4.01 True 

4.1 False 



In [52]:
A = Vertex('A', -1.6169099810000001, -1.0946099760000001, 11.25370026)
B = Vertex('B', -1.801380038, -1.165879965, 10.89350033)
C = Vertex('C', -1.599470019, -0.90017002800000001, 11.087599750000001)

edges = [Edge(1, A, B), Edge(2, B, C), Edge(3, C, A)]
face = Face(0, edges)
print(face)
print([v.id for v in face.vertices])

v1 = Vertex(1, -2.076020002, -3.637000084, 10.35280037)
print(v1.y, face.is_in_line_with(v1), '\n')

v1 = Vertex(1, 0, 3.99, -0.1)
print(v1.y, face.is_in_line_with(v1), '\n')

v1 = Vertex(1, 0, 4.00, -0.0)
print(v1.y, face.is_in_line_with(v1), '\n')

v1 = Vertex(1, 0, 4.01, .1)
print(v1.y, face.is_in_line_with(v1), '\n')

v1 = Vertex(1, 0, 4.10, -0.1)
print(v1.y, face.is_in_line_with(v1), '\n')

[-1.616909981, -1.094609976, 11.25370026]
[-1.599470019, -0.900170028, 11.08759975]
[-1.801380038, -1.165879965, 10.89350033]
['A', 'C', 'B']


AttributeError: 'Face' object has no attribute 'is_in_line_with'

In [57]:
import random
A = Vertex('A', -4., 4., 0)
B = Vertex('B', 4., 4., 0)
C = Vertex('C', 4., -4., 0)
#D = Vertex('D', -4., -4., 0)
e1 = Edge(1, A, B)
e2 = Edge(2, B, C)
e3 = Edge(3, C, A)
#e4 = Edge(4, D, A)
# valid_paths = [
#     [A, B, C, D], [B, C, D, A], [C, D, A, B], [D, A, B, C],
#     [A, D, C, B], [D, C, B, A], [C, B, A, D], [B, A, D, C]
# ]
valid_paths = [
    [A, B, C], [B, C, A], [C, A, B],
    [A, C, B], [C, B, A], [B, A, C]
]
edges = [e1, e2, e3]
for _ in range(50):
    random.shuffle(edges)
    face = Face(1, edges)
    print([e.id for e in edges], "=>", [v.id for v in face.vertices])
    assert(face.vertices in valid_paths)

[2, 1, 3] => ['B', 'A', 'C']
[3, 2, 1] => ['C', 'B', 'A']
[1, 3, 2] => ['A', 'C', 'B']
[2, 1, 3] => ['B', 'A', 'C']
[3, 1, 2] => ['C', 'B', 'A']
[1, 3, 2] => ['A', 'C', 'B']
[3, 2, 1] => ['C', 'B', 'A']
[3, 2, 1] => ['C', 'B', 'A']
[1, 2, 3] => ['A', 'C', 'B']
[1, 2, 3] => ['A', 'C', 'B']
[1, 2, 3] => ['A', 'C', 'B']
[2, 3, 1] => ['B', 'A', 'C']
[1, 2, 3] => ['A', 'C', 'B']
[1, 3, 2] => ['A', 'C', 'B']
[1, 3, 2] => ['A', 'C', 'B']
[3, 2, 1] => ['C', 'B', 'A']
[2, 3, 1] => ['B', 'A', 'C']
[1, 3, 2] => ['A', 'C', 'B']
[2, 1, 3] => ['B', 'A', 'C']
[3, 2, 1] => ['C', 'B', 'A']
[2, 3, 1] => ['B', 'A', 'C']
[1, 3, 2] => ['A', 'C', 'B']
[2, 3, 1] => ['B', 'A', 'C']
[1, 3, 2] => ['A', 'C', 'B']
[3, 1, 2] => ['C', 'B', 'A']
[1, 3, 2] => ['A', 'C', 'B']
[2, 3, 1] => ['B', 'A', 'C']
[3, 1, 2] => ['C', 'B', 'A']
[1, 3, 2] => ['A', 'C', 'B']
[1, 2, 3] => ['A', 'C', 'B']
[2, 3, 1] => ['B', 'A', 'C']
[2, 1, 3] => ['B', 'A', 'C']
[2, 3, 1] => ['B', 'A', 'C']
[2, 1, 3] => ['B', 'A', 'C']
[1, 3, 2] => [

In [6]:
A = Vertex(1, 4, 4, 0)
B = Vertex(2, -4, 4, 0)
V = Vertex(3, -3.9, 0, 0)

integral(A, B, V)

1.1191327043588686

In [62]:
v1 = Vertex(1, -4., -4., 0)
v2 = Vertex(2, 4., -4., 0)
v3 = Vertex(3, -4., 4., 0)
v4 = Vertex(4, 4., 4., 0)

face = Face(0, [v1, v2, v3, v4])

v1 = Vertex(1, 0, 3.99, -0.1)
print(v1.z, face.is_behind(v1), '\n')

v1 = Vertex(1, 0, 4.00, -0.0)
print(v1.z, face.is_behind(v1), '\n')

v1 = Vertex(1, 0, 4.01, .1)
print(v1.z, face.is_behind(v1), '\n')

v1 = Vertex(1, 0, 4.10, -0.1)
print(v1.z, face.is_behind(v1), '\n')



AttributeError: 'Vertex' object has no attribute 'v1'

In [6]:
face = faces[36]
v0 = face.vertices[0]
print(face, '\n')
print("v0", v0, face.is_blocking(v0))

v1 = Vertex(1, -1.6169099810000001, -1.0946099760000001, 0.)
print('v1', v1, face.is_blocking(v1))

v749 = vertices[749]
print('\nv749', v749, face.is_blocking(v749))
v749.z -= 5.
for mod in range(10):
    v749.z += 1.
    print('v749', v749, face.is_blocking(v749))
    

[-1.6169099810000001, -1.0946099760000001, 11.25370026]
[-1.801380038, -1.165879965, 10.89350033]
[-1.599470019, -0.90017002800000001, 11.087599750000001] 

v0 [-1.6169099810000001, -1.0946099760000001, 11.25370026] False
v1 [-1.616909981, -1.094609976, 0.0] False
is_behind
A [ -1.61690998  -1.09460998  11.25370026]
B [ -1.80138004  -1.16587997  10.89350033]
C [ -1.59947002  -0.90017003  11.08759975]
AB [-0.18447006 -0.07126999 -0.36019993]
AC [ 0.01743996  0.19443995 -0.16610051]
ABAC [ 0.08187524 -0.03692244 -0.0346254 ]
numer -0.387674023135
demon 0.277114369022
result -1.39896759776
is_behind False

v749 [-1.653570056, -1.122009993, -8.0032100679999996] False
is_behind
A [ -1.61690998  -1.09460998  11.25370026]
B [ -1.80138004  -1.16587997  10.89350033]
C [ -1.59947002  -0.90017003  11.08759975]
AB [-0.18447006 -0.07126999 -0.36019993]
AC [ 0.01743996  0.19443995 -0.16610051]
ABAC [ 0.08187524 -0.03692244 -0.0346254 ]
numer -0.387674023135
demon 0.415615978585
result -0.93276977573

In [4]:
face = faces[656]
vertex = vertices[449]
print("Vertex")
print(vertex)
print("Face")
print(face.vertices)
print("\nThis face is blocking this vertex?")
print(face.is_blocking(vertex))

Vertex
[4.5775899889999998, -12.382800100000001, -0.36562800400000001]
Face
[[-6.0531401630000001, -4.8197398189999996, 6.010980129], [-5.5431098939999996, -6.2721600530000003, 5.9813899990000001], [-4.9101099970000002, -5.644229889, 7.4963197709999996], [-5.2279100420000004, -4.6442799570000002, 7.5121898649999999]]

This face is blocking this vertex?
False


In [5]:
print("Vertex: ", vertices[0])
print("Edge: ", edges[0])
print("Face:\n", faces[0])

Vertex:  [1.4822599889999999, -9.4975204469999994, 8.6801795960000003]
Edge:  [[1.4822599889999999, -9.4975204469999994, 8.6801795960000003], [1.045789957, -9.7227897639999998, 6.7623200419999998]]
Face:
 [1.4822599889999999, -9.4975204469999994, 8.6801795960000003]
[2.3399600980000002, -9.1642999649999997, 8.2300500870000004]
[1.045789957, -9.7227897639999998, 6.7623200419999998]


In [34]:
vertex = vertices[172]
print("Vertex", vertex)
for fid in faces.keys():
    face = faces[fid]
    if face.is_blocking(vertex):
        print("This face is blocking:", fid)
        print(face)


Vertex [-1.6169099810000001, -1.0946099760000001, 11.25370026]
is_behind
A [  1.93268001   2.68939996 -10.09669971]
B [ 1.67683005 -1.12414002 -8.00249958]
C [ 0.01165    -0.980479   -8.33932972]
AB [-0.25584996 -3.81353998  2.09420013]
AC [-1.92103001 -3.66987896  1.75736999]
ABAC [ 0.98366028 -3.57339826 -6.38698638]
numer 54.4572996398
demon -71.8772302378
result -0.757643268384
is_behind False
is_behind
A [-1.26735997  1.47167003  9.67045975]
B [ -1.46402001  -0.223911    10.58269978]
C [-2.0968101  -0.63485402  9.81933975]
AB [-0.19666004 -1.69558103  0.91224003]
AC [-0.82945013 -2.10652405  0.14888   ]
ABAC [ 1.66921746 -0.72737887 -0.9921308 ]
numer -10.877543837
demon -11.1651426502
result 0.974241366889
is_behind True
This face is blocking: 489
[-1.2673599719999999, 1.471670032, 9.6704597470000007]
[-1.4640200139999999, -0.223911002, 10.58269978]
[-2.0968101020000001, -0.63485401900000005, 9.8193397519999994]
[-1.725489974, 1.1278699640000001, 9.3631601329999992]
is_behind
A [

In [61]:
face = faces[36]
print("Face")
print(face.vertices)

for i in range(700, 800):
    vertex = vertices[i]
    print("Vertex", vertex.id, face.is_blocking(vertex))

Face
[[-1.6169099810000001, -1.0946099760000001, 11.25370026], [-1.801380038, -1.165879965, 10.89350033], [-1.599470019, -0.90017002800000001, 11.087599750000001]]
Vertex 700 False
Vertex 701 False
Vertex 702 False
Vertex 703 False
Vertex 704 False
Vertex 705 False
Vertex 706 False
Vertex 707 False
Vertex 708 False
Vertex 709 False
Vertex 710 False
Vertex 711 False
Vertex 712 False
Vertex 713 False
Vertex 714 False
Vertex 715 False
Vertex 716 False
Vertex 717 False
Vertex 718 False
Vertex 719 False
Vertex 720 False
Vertex 721 False
Vertex 722 False
Vertex 723 False
Vertex 724 False
Vertex 725 False
Vertex 726 False
Vertex 727 False
Vertex 728 False
Vertex 729 False
Vertex 730 False
Vertex 731 False
Vertex 732 False
Vertex 733 False
Vertex 734 False
Vertex 735 False
Vertex 736 False
Vertex 737 False
Vertex 738 False
Vertex 739 False
Vertex 740 False
Vertex 741 False
Vertex 742 False
Vertex 743 False
Vertex 744 False
Vertex 745 False
Vertex 746 False
Vertex 747 False
Vertex 748 False
Ver

KeyError: 775